### Chunking Semántico (Semantic Chunking)
- SemanticChunker es un divisor de documentos que utiliza la similitud de embeddings entre oraciones para decidir los límites de los fragmentos (chunks).

- Asegura que cada fragmento sea semánticamente coherente y no se corte a mitad de una idea, como sucede con los divisores tradicionales basados en caracteres/tokens.

In [3]:
# Importar el modelo de transformers para generar embeddings de oraciones
from sentence_transformers import SentenceTransformer
# Importar función para calcular similitud coseno entre vectores
from sklearn.metrics.pairwise import cosine_similarity
# Importar numpy para operaciones numéricas
import numpy as np

In [6]:
## Inicializar el modelo de embeddings pre-entrenado (384 dimensiones)
model = SentenceTransformer('all-MiniLM-L6-v2')

## Texto de ejemplo con diferentes temas
text = """
LangChain es un framework para crear aplicaciones con LLM.
Langchain proporciona abstracciones modulares para combinar LLM con herramientas como OpenAI y Pinecone.
Permite crear cadenas, agentes, memoria y recuperadores.
La Torre Eiffel se encuentra en París.
Francia es un destino turístico popular.
"""

## Paso 1: Dividir el texto en oraciones individuales
# Eliminar líneas vacías y espacios en blanco
sentences = [s.strip() for s in text.split("\n") if s.strip()]

## Paso 2: Generar embeddings (vectores) para cada oración
# Cada oración se convierte en un vector numérico de 384 dimensiones
embeddings = model.encode(sentences)

# Paso 3: Inicializar parámetros para el chunking semántico
threshold = 0.7  # Umbral de similitud: controla qué tan relacionadas deben estar las oraciones (0-1)
chunks = []  # Lista para almacenar los fragmentos finales
current_chunk = [sentences[0]]  # Comenzar con la primera oración

## Paso 4: Agrupación semántica basada en el umbral de similitud

for i in range(1, len(sentences)):
    # Calcular similitud coseno entre la oración actual y la anterior
    # Devuelve un valor entre -1 y 1 (1 = idénticas, 0 = sin relación)
    sim = cosine_similarity(
        [embeddings[i - 1]],  # Embedding de la oración anterior
        [embeddings[i]]        # Embedding de la oración actual
    )[0][0]

    # Si la similitud es mayor o igual al umbral, agregar a chunk actual
    if sim >= threshold:
        current_chunk.append(sentences[i])
    else:
        # Si no hay suficiente similitud, guardar el chunk actual y empezar uno nuevo
        chunks.append(" ".join(current_chunk))
        current_chunk = [sentences[i]]

# Agregar el último chunk a la lista
chunks.append(" ".join(current_chunk))

# Mostrar los chunks resultantes
print("\n📌 Chunks Semánticos:")
for idx, chunk in enumerate(chunks):
    print(f"\nChunk {idx+1}:\n{chunk}")


📌 Chunks Semánticos:

Chunk 1:
LangChain es un framework para crear aplicaciones con LLM.

Chunk 2:
Langchain proporciona abstracciones modulares para combinar LLM con herramientas como OpenAI y Pinecone.

Chunk 3:
Permite crear cadenas, agentes, memoria y recuperadores.

Chunk 4:
La Torre Eiffel se encuentra en París.

Chunk 5:
Francia es un destino turístico popular.


### Pipeline RAG con Código Modular

In [7]:
# Importar modelo de sentence transformers para embeddings
from sentence_transformers import SentenceTransformer
# Importar función para calcular similitud entre vectores
from sklearn.metrics.pairwise import cosine_similarity
# Importar clase Document de LangChain para estructurar documentos
from langchain.schema import Document
# Importar FAISS para crear base de datos vectorial en memoria
from langchain.vectorstores import FAISS
# Importar embeddings de OpenAI
from langchain_openai import OpenAIEmbeddings
# Importar función para inicializar modelos de chat
from langchain.chat_models import init_chat_model
# Importar componentes para crear cadenas ejecutables
from langchain.schema.runnable import RunnableLambda, RunnableMap
# Importar clase para crear templates de prompts
from langchain.prompts import PromptTemplate
# Importar parser para convertir salida del LLM a string
from langchain_core.output_parsers import StrOutputParser
# Importar librería para manejo de variables de entorno
import os
# Configurar API key de Groq desde archivo .env
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [10]:
### Chunker Semántico Personalizado con Umbral de Similitud

class ThresholdSematicChunker:
    """
    Clase que divide textos en fragmentos semánticamente coherentes
    basándose en la similitud entre oraciones
    """
    def __init__(self, model_name="all-MiniLM-L6-v2", threshold=0.7):
        # Inicializar el modelo de embeddings
        self.model = SentenceTransformer(model_name)
        # Definir el umbral de similitud (0-1)
        self.threshold = threshold 

    def split(self, text: str):
        """
        Divide un texto en chunks basándose en similitud semántica
        
        Args:
            text: Texto a dividir
            
        Returns:
            Lista de strings (chunks)
        """
        # Dividir texto en oraciones usando el punto como separador
        sentences = [s.strip() for s in text.split('.') if s.strip()]
        # Generar embeddings para todas las oraciones
        embeddings = self.model.encode(sentences)
        # Inicializar lista de chunks y chunk actual
        chunks = []
        current_chunk = [sentences[0]]

        # Iterar sobre las oraciones comparando similitudes
        for i in range(1, len(sentences)):
            # Calcular similitud coseno entre oración actual y anterior
            sim = cosine_similarity([embeddings[i - 1]], [embeddings[i]])[0][0]
            # Si similitud >= umbral, agregar a chunk actual
            if sim >= self.threshold:
                current_chunk.append(sentences[i])
            else:
                # Si no, guardar chunk actual y empezar uno nuevo
                chunks.append(". ".join(current_chunk) + ".")
                current_chunk = [sentences[i]]

        # Agregar el último chunk
        chunks.append(". ".join(current_chunk) + ".")
        return chunks
    
    def split_documents(self, docs):
        """
        Divide una lista de documentos de LangChain en chunks semánticos
        
        Args:
            docs: Lista de objetos Document
            
        Returns:
            Lista de objetos Document con chunks
        """
        result = []
        # Procesar cada documento
        for doc in docs:
            # Dividir contenido en chunks y crear nuevos Documents
            for chunk in self.split(doc.page_content):
                # Preservar los metadatos originales del documento
                result.append(Document(page_content=chunk, metadata=doc.metadata))

        return result

In [11]:
# Texto de ejemplo con diferentes temas semánticos
sample_text = """
LangChain es un framework para crear aplicaciones con LLM.
Langchain proporciona abstracciones modulares para combinar LLM con herramientas como OpenAI y Pinecone.
Permite crear cadenas, agentes, memoria y recuperadores.
La Torre Eiffel se encuentra en París.
Francia es un destino turístico popular.
"""

# Crear un objeto Document de LangChain con el texto
doc = Document(page_content=sample_text)
# Mostrar el documento
doc

Document(metadata={}, page_content='\nLangChain es un framework para crear aplicaciones con LLM.\nLangchain proporciona abstracciones modulares para combinar LLM con herramientas como OpenAI y Pinecone.\nPermite crear cadenas, agentes, memoria y recuperadores.\nLa Torre Eiffel se encuentra en París.\nFrancia es un destino turístico popular.\n')

In [12]:
### Aplicar Chunking Semántico

# Crear instancia del chunker con umbral de 0.7 (70% de similitud mínima)
chunker = ThresholdSematicChunker(threshold=0.7)
# Dividir el documento en chunks semánticamente coherentes
chunks = chunker.split_documents([doc])
# Mostrar los chunks resultantes
chunks

[Document(metadata={}, page_content='LangChain es un framework para crear aplicaciones con LLM.'),
 Document(metadata={}, page_content='Langchain proporciona abstracciones modulares para combinar LLM con herramientas como OpenAI y Pinecone.'),
 Document(metadata={}, page_content='Permite crear cadenas, agentes, memoria y recuperadores.'),
 Document(metadata={}, page_content='La Torre Eiffel se encuentra en París.'),
 Document(metadata={}, page_content='Francia es un destino turístico popular.')]

In [13]:
### Crear Vector Store (Base de Datos Vectorial)

# Importar librería para manejo de variables de entorno
import os
# Configurar API key de OpenAI desde archivo .env
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
# Inicializar el modelo de embeddings de OpenAI
embedding = OpenAIEmbeddings()
# Crear base de datos vectorial FAISS a partir de los chunks
# FAISS indexa los embeddings para búsqueda rápida por similitud
vectorstore = FAISS.from_documents(chunks, embedding)
# Convertir el vectorstore en un retriever para búsqueda de documentos relevantes
retriever = vectorstore.as_retriever()

In [14]:
## Template de Prompt para RAG

# Crear template que define cómo formatear la información para el LLM
# {context} será reemplazado por documentos recuperados
# {question} será reemplazado por la pregunta del usuario
template = """Responde la pregunta basándote en el siguiente contexto:

{context}

Pregunta: {question}
"""

# Crear objeto PromptTemplate a partir del template
prompt = PromptTemplate.from_template(template)
# Mostrar el prompt
prompt

PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='Responde la pregunta basándote en el siguiente contexto:\n\n{context}\n\nPregunta: {question}\n')

In [19]:
## Inicializar el Modelo de Lenguaje (LLM)

# Inicializar modelo Gemma2-9b-it de Groq con temperatura 0.4
# Temperatura baja = respuestas más deterministas y precisas
llm = init_chat_model(model="groq:llama-3.1-8b-instant", temperature=0.4)

### Crear Cadena RAG usando LCEL (LangChain Expression Language)

rag_chain = (
    # RunnableMap ejecuta múltiples funciones en paralelo
    RunnableMap(
        {
        # Lambda para recuperar documentos relevantes usando el retriever
        "context": lambda x: retriever.invoke(x["question"]),
        # Lambda para pasar la pregunta sin modificar
        "question": lambda x: x["question"],  
        }
    )
    # Operador | encadena componentes secuencialmente
    | prompt  # Formatear contexto y pregunta en el template
    | llm  # Enviar prompt al modelo de lenguaje
    | StrOutputParser()  # Convertir respuesta del LLM a string
)

# Ejecutar consulta de ejemplo
query = {"question": "¿Para qué se utiliza LangChain?"}
# Invocar la cadena RAG con la pregunta
result = rag_chain.invoke(query)

# Mostrar la respuesta generada
print(result)

Según el contexto, LangChain se utiliza para crear aplicaciones con LLM (Modelos de Lenguaje de Llama).


### Chunker Semántico con LangChain (Implementación Nativa)

In [20]:
# Importar embeddings de OpenAI
from langchain_openai import OpenAIEmbeddings
# Importar SemanticChunker experimental de LangChain
from langchain_experimental.text_splitter import SemanticChunker
# Importar cargador de archivos de texto
from langchain.document_loaders import TextLoader

In [23]:
## Cargar el documento desde archivo de texto
loader = TextLoader("langchain_intro.txt")
# Cargar contenido del archivo
docs = loader.load()

## Inicializar modelo de embeddings de OpenAI
embedding = OpenAIEmbeddings()

## Crear el chunker semántico de LangChain
# Usa internamente embeddings para detectar cambios semánticos
chunker = SemanticChunker(embedding)

## Dividir los documentos en chunks semánticos
chunks = chunker.split_documents(docs)

## Mostrar resultados

# Iterar sobre cada chunk generado
for i, chunk in enumerate(chunks):
    # Imprimir número de chunk y su contenido
    print(f"\n chunk {i+1}:\n{chunk.page_content}")


 chunk 1:
LangChain es un framework para crear aplicaciones con LLM. Langchain proporciona abstracciones modulares para combinar LLM con herramientas como OpenAI y Pinecone. Permite crear cadenas, agentes, memoria y recuperadores.

 chunk 2:
La Torre Eiffel se encuentra en ParÃ­s. Francia es un destino turÃ­stico popular.
